In [1]:
import gmaps
# working with gmaps
with open('api.txt') as f:
    API_KEY = f.readline()
    f.close
gmaps.configure(api_key = API_KEY)

jakarta_coord = (-6.175252901681506, 106.82713921415032)
gmaps.figure(center=jakarta_coord, zoom_level=16)

Figure(layout=FigureLayout(height='420px'))